In [1]:
import torch
import numpy as np
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import os
from flask import Flask, jsonify, request
from flask_restful import Api, Resource

I0816 22:19:36.185438 10724 file_utils.py:39] PyTorch version 1.4.0 available.


In [2]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

I0816 22:19:57.175183 10724 filelock.py:274] Lock 2140732403048 acquired on C:\Users\samar/.cache\torch\transformers\26561bc9e840d8945f475d0d4c4b9df32025eadd79894b867b570cb1d09e67a9.3817cc1260a6b941b17af62b4f2a942b9825f209d8e2eed99e79e96f85f59aab.lock
I0816 22:19:57.178174 10724 file_utils.py:748] https://s3.amazonaws.com/models.huggingface.co/bert/t5-small-config.json not found in cache or force_download set to True, downloading to C:\Users\samar\.cache\torch\transformers\tmpodb_gh07


I0816 22:19:58.049338 10724 file_utils.py:752] storing https://s3.amazonaws.com/models.huggingface.co/bert/t5-small-config.json in cache at C:\Users\samar/.cache\torch\transformers\26561bc9e840d8945f475d0d4c4b9df32025eadd79894b867b570cb1d09e67a9.3817cc1260a6b941b17af62b4f2a942b9825f209d8e2eed99e79e96f85f59aab
I0816 22:19:58.054325 10724 file_utils.py:755] creating metadata file for C:\Users\samar/.cache\torch\transformers\26561bc9e840d8945f475d0d4c4b9df32025eadd79894b867b570cb1d09e67a9.3817cc1260a6b941b17af62b4f2a942b9825f209d8e2eed99e79e96f85f59aab
I0816 22:19:58.060310 10724 filelock.py:318] Lock 2140732403048 released on C:\Users\samar/.cache\torch\transformers\26561bc9e840d8945f475d0d4c4b9df32025eadd79894b867b570cb1d09e67a9.3817cc1260a6b941b17af62b4f2a942b9825f209d8e2eed99e79e96f85f59aab.lock
I0816 22:19:58.062303 10724 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/t5-small-config.json from cache at C:\Users\samar/.cache\

I0816 22:21:00.450437 10724 file_utils.py:752] storing https://cdn.huggingface.co/t5-small-pytorch_model.bin in cache at C:\Users\samar/.cache\torch\transformers\9b662cba85524bef76fff5eb77d767407ac36f3fe492869331c011efd2b3a082.388aab7f5c8ed273dc71eb98334d76a3caf1b3280b476c1c77fba861c65445f3
I0816 22:21:00.453430 10724 file_utils.py:755] creating metadata file for C:\Users\samar/.cache\torch\transformers\9b662cba85524bef76fff5eb77d767407ac36f3fe492869331c011efd2b3a082.388aab7f5c8ed273dc71eb98334d76a3caf1b3280b476c1c77fba861c65445f3
I0816 22:21:00.456424 10724 filelock.py:318] Lock 2140732403048 released on C:\Users\samar/.cache\torch\transformers\9b662cba85524bef76fff5eb77d767407ac36f3fe492869331c011efd2b3a082.388aab7f5c8ed273dc71eb98334d76a3caf1b3280b476c1c77fba861c65445f3.lock
I0816 22:21:00.458416 10724 modeling_utils.py:667] loading weights file https://cdn.huggingface.co/t5-small-pytorch_model.bin from cache at C:\Users\samar/.cache\torch\transformers\9b662cba85524bef76fff5eb77d767

cpu


In [3]:
# INSERT TEXT HERE

text ="""
coronavirus vaccine developed by the University of Oxford appears safe and triggers an immune response. Trials involving 1,077 people showed the injection led to them making antibodies and T-cells that can fight coronavirus. The findings are hugely promising, but it is still too soon to know if this is enough to offer protection and larger trials are under way.
The UK has already ordered 100 million doses of the vaccine. How does the vaccine work?
The vaccine - called ChAdOx1 nCoV-19 - is being developed at unprecedented speed. It is made from a genetically engineered virus that causes the common cold in chimpanzees.
It has been heavily modified, first so it cannot cause infections in people and also to make it "look" more like coronavirus. Scientists did this by transferring the genetic instructions for the coronavirus's "spike protein" - the crucial tool it uses to invade our cells - to the vaccine they were developing. This means the vaccine resembles the coronavirus and the immune system can learn how to attack it. What are antibodies and T-cells? Much of the focus on coronavirus so far has been about antibodies, but these are only one part of our immune defence. Antibodies are small proteins made by the immune system that stick onto the surface of viruses. Neutralising antibodies can disable the coronavirus. T-cells, a type of white blood cell, help co-ordinate the immune system and are able to spot which of the body's cells have been infected and destroy them. Nearly all effective vaccines induce both an antibody and a T-cell response. Sample from patients are analysed as part of the trial.

Levels of T-cells peaked 14 days after vaccination and antibody levels peaked after 28 days. The study has not run for long enough to understand how long they may last, the study in the Lancet showed. Prof Andrew Pollard, from the Oxford research group told the BBC: "We're really pleased with the results published today as we're seeing both neutralising antibodies and T-cells." They're extremely promising and we believe the type of response that may be associated with protection. "But the key question everyone wants to know is does the vaccine work, does it offer protection and we're in a waiting game."

The study showed 90% of people developed neutralising antibodies after one dose. Only ten people were given two doses and all of them produced neutralising antibodies. 
"We don't know the level needed for protection, but we can maximise responses with a second dose," Prof Pollard told the BBC. Is it safe? Yes, but there are side-effects.

There were no dangerous side-effects from taking the vaccine, however, 70% of people on the trial developed either fever or headache. The researchers say this could be managed with paracetamol. Prof Sarah Gilbert, from the University of Oxford, UK, says: "There is still much work to be done before we can confirm if our vaccine will help manage the Covid-19 pandemic, but these early results hold promise."

What are the next steps in the trial? The results so far are promising, but their main purpose is to ensure the vaccine is safe enough to give to people. The study cannot show whether the vaccine can either prevent people from becoming ill or even lessen their symptoms of Covid-19. More than 10,000 people will take part in the next stage of the trials in the UK.

However, the trial has also been expanded to other countries because levels of coronavirus are low in the UK, making it hard to know if the vaccine is effective.
There will be a large trial involving 30,000 people in the US as well 2,000 in South Africa and 5,000 in Brazil. There are also calls to perform "challenge trials" in which vaccinated people are deliberately infected with coronavirus. However, there are ethical concerns due to a lack of treatments. When will I get a vaccine? It is possible a coronavirus vaccine will be proven effective before the end of the year, however, it will not be widely available. Health and care workers will be prioritised as will people who are deemed at high risk from Covid-19 due to their age or medical conditions. However, widespread vaccination is likely to be, at the earliest, next year even if everything goes to plan. Boris Johnson said: "Obviously I'm hopeful, I've got my fingers crossed, but to say I'm 100% confident we'll get a vaccine this year, or indeed next year, is, alas, just an exaggeration. "We're not there yet."
What progress is being made with other vaccines?
The Oxford vaccine is not the first to reach this stage, with groups in the US and China also publishing similar results. The US company Moderna was first out of the blocks and its vaccine can produce neutralising antibodies. They are injecting coronavirus RNA (its genetic code), which then starts making viral proteins in order to trigger an immune response. The companies BioNtech and Pfizer have also had positive results using their RNA vaccine. A technique similar to the Oxford one, developed in China, also seems promising. However, all these approaches are at the absolute boundary of science and have not been proven to work before. More traditional methods of vaccine development are also being investigated. The company Valneva is taking the whole coronavirus, inactivating it and then inject it. In total there are 23 coronavirus vaccines in clinical trials around the world and another 140 in early stage development.
"""
print(device)
print(len(text))


cpu
5416


In [4]:
text[10:2021]

'us vaccine developed by the University of Oxford appears safe and triggers an immune response. Trials involving 1,077 people showed the injection led to them making antibodies and T-cells that can fight coronavirus. The findings are hugely promising, but it is still too soon to know if this is enough to offer protection and larger trials are under way.\nThe UK has already ordered 100 million doses of the vaccine. How does the vaccine work?\nThe vaccine - called ChAdOx1 nCoV-19 - is being developed at unprecedented speed. It is made from a genetically engineered virus that causes the common cold in chimpanzees.\nIt has been heavily modified, first so it cannot cause infections in people and also to make it "look" more like coronavirus. Scientists did this by transferring the genetic instructions for the coronavirus\'s "spike protein" - the crucial tool it uses to invade our cells - to the vaccine they were developing. This means the vaccine resembles the coronavirus and the immune syst

In [ ]:
%tb
app = Flask(__name__)
api = Api(app)

class SendSummary(Resource):
    @staticmethod
    def post():
#         text = "coronavirus vaccine developed by the University of Oxford appears safe and triggers an immune response.\n\nTrials involving 1,077 people showed the injection led to them making antibodies and T-cells that can fight coronavirus.\n\nThe findings are hugely promising, but it is still too soon to know if this is enough to offer protection and larger trials are under way.\n\nThe UK has already ordered 100 million doses of the vaccine.\n\nHow does the vaccine work?\nThe vaccine - called ChAdOx1 nCoV-19 - is being developed at unprecedented speed.\n\nIt is made from a genetically engineered virus that causes the common cold in chimpanzees.\n\nIt has been heavily modified, first so it cannot cause infections in people and also to make it look more like coronavirus.\n\nScientists did this by transferring the genetic instructions for the coronavirus\'s spike protein - the crucial tool it uses to invade our cells - to the vaccine they were developing.\n\nThis means the vaccine resembles the coronavirus and the immune system can learn how to attack it.\n\nWhat are antibodies and T-cells?\nMuch of the focus on coronavirus so far has been about antibodies, but these are only one part of our immune defence.\n\nAntibodies are small proteins made by the immune system that stick onto the surface of viruses.\n\nNeutralising antibodies can disable the coronavirus.\n\nT-cells, a type of white blood cell, help co-ordinate the immune system and are able to spot which of the body\'s cells have been infected and destroy them.\n\nNearly all effective vaccines induce both an antibody and a T-cell response.\n\nImage copyrightOXFORD UNIVERSITY\nImage caption\nSample from patients are analysed as part of the trial.\nLevels of T-cells peaked 14 days after vaccination and antibody levels peaked after 28 days."
        
        posted_data = request.get_json()
        text = posted_data["req"].strip().replace("\n","")
        text_toks = text.split(" ")
        l = len(text_toks)
        preprocess_text = text.strip().replace("\n","")
        t5_prepared_Text = "summarize: "+ preprocess_text
        print(t5_prepared_Text)
        print("\n")
        tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
        
        model.to(device)
        
        summary_ids = model.generate(tokenized_text,
                                            num_beams=5,
                                            no_repeat_ngram_size=2,
                                            min_length= abs(int(l * 0.3)),
                                            max_length= abs(int(l * 0.5)),
                                            early_stopping=True)
        
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        output_texts = output.split('.')
        response = ""
        
        for x in range(len(output_texts)-1):
            response += ('-> ' + output_texts[x].strip()) + "\n"
        
        print ("Summarized text: \n" + response)
        
        return jsonify({
            'resp': response
        })

api.add_resource(SendSummary, '/resp')
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

No traceback available to show.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


I0816 22:23:23.776479 10724 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [6]:
# preprocess_text = text.strip().replace("\n","")
# t5_prepared_Text = "summarize: "+preprocess_text
# # print ("original text preprocessed: \n", preprocess_text)
# txt_length = len(preprocess_text)
# print(txt_length)
# max_length_output = (int)(np.floor(txt_length * 0.2))
# print(max_length_output)

In [7]:
# tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
# text_toks = text.split(" ")
# l = len(text_toks)

In [8]:
# %%time
# # summmarize 
# model.to(device)
# summary_ids = model.generate(tokenized_text,
#                                     num_beams=5,
#                                     no_repeat_ngram_size=2,
#                                     min_length= int(l * 0.7),
#                                     max_length= int(l * 0.8),
#                                     early_stopping=False)

# output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# print(output)